[View in Colaboratory](https://colab.research.google.com/github/vigneshrao98/Colab-Keras-CNN/blob/master/Keras_ShallowCNN.ipynb)

#Introduction
Welcome! This is a Google Colab notebook. 

You can rename this notebook by clicking on 'name.ipynb' above.
In this notebook you can find out how to make and train a simple keras model using the Colab's free K80 gpu!

#Setting the runtime
To use GPU accelerated runtime, just follow 

*Runtime > change runtime type > and select GPU as hardware exelerator*

*Additonaly you must have noticed you can also chose which version of python you want to compile in!*


#The Setup

You will need to do the following to steps to instal te required libraries and perform authorisation-

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Click the link, copy verification code and paste it to text box.
This will complete the authorisation!

Now you will mount your Google Drive to make it work with colab.

Exectue the following code-

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

#The Directories
You will often need to upload data that you want to use while executing or training like a  `Utility file` or `Training/Test data`.

This import will get the required files for doing so-

In [0]:
from google.colab import files

This line will list the files in your directory-

In [8]:
files.os.listdir()

['datalab',
 'cifar-10-batches-py',
 '.config',
 'distutils.py',
 'distutils (1).py',
 '.cache',
 'Transfer-Learning-in-keras---custom-data',
 '.forever',
 '.local',
 '__pycache__',
 'Parameters.py',
 'distutils3.py',
 '.ipython',
 '.keras',
 '.nv']

This will let you upload files-

In [1]:
files.upload()

NameError: ignored

And this for downloading files (replace file-name with the required file name) -

In [0]:
files.download('file-name')

Now that we are done with the basics lets get started on building the model. But first lets install the Keras library-

In [0]:
!pip install keras

##Simple CNN on the CIFAR-10 dataset
Following is a implementation of Convolutional Neural Networks in Keras framework. It is a shallow network having only 4 Conv, 2 MaxPool and 2 Fully connected dense layers. As the model complexity is low, regularization is kept to a minimum. 

For implementing the model we will need a few utility functions present in a Utility file.

We will need to upload it to the directory by following the steps given above for uploading. The file *distutils.py* is te one needed to be uploaded and it  is available on the Github repo.
Link:

This will help you understand how uploading works on Colab :D

In [11]:
from google.colab import files
files.upload()

Saving utils.py to utils.py


{'utils.py': b'#from sklearn.datasets import fetch_mldata\r\nfrom sklearn.preprocessing import OneHotEncoder\r\nfrom matplotlib import pyplot\r\nfrom scipy.misc import toimage\r\n#from sklearn.model_selection import train_test_split\r\nfrom urllib.request import urlretrieve\r\nfrom keras.preprocessing.image import ImageDataGenerator\r\nimport numpy as np\r\nimport os\r\nimport tarfile\r\nimport pickle\r\n\r\ndef lrScheduler(epoch):\r\n    lrate = 0.001\r\n    if epoch > 75:\r\n        lrate = 0.0005\r\n    elif epoch > 100:\r\n        lrate = 0.0003        \r\n    return lrate\r\n\r\ndef displayImg(X):\r\n    pyplot.figure(1)\r\n    k = 0\r\n    for i in range(0,4):\r\n        for j in range(0,4):\r\n            pyplot.subplot2grid((4,4),(i,j))\r\n            pyplot.imshow(toimage(X[k]))\r\n            k = k+1\r\n    # show the plot\r\n    pyplot.show()\r\n\r\ndef unPickle(src):\r\n    data = pickle.load(open(src, \'rb\'), encoding=\'latin1\')\r\n    return data\r\n\r\ndef downloadData

Once you are done with uploading the files you can start with importing the required libraries-

In [0]:
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Dense
from keras import regularizers
from keras.optimizers import rmsprop,SGD
from keras.callbacks import LearningRateScheduler

from utils import getCifarDataset

#Hyperparameters
EPOCHS = 10
BATCHSIZE = 64
LR = 0.01
MOMENTUM = 0.9



Now we will download the CIFAR-10 dataset and get it into training and testing files!

Feel free to check out how function *getCifarDataset( )* works by checking it out in *distutils.py*

In [15]:
x_train, x_test, y_train, y_test = getCifarDataset()
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)


Preparing train and test sets
Preparing test set...
Done.
(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)
float32 float32 int32 int32


Now let's define the model-

In [0]:
def createModel():
  model = Sequential()
    
  model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
  model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
  model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  return model

Initializing the model-

In [0]:
mod = createModel()

A funtion to compile the model-

In [0]:
def compile_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = SGD(LR, MOMENTUM),
        metrics = ['accuracy'])
    return m

Compiling the model-

In [0]:
model = compile_model(mod)

Fitting the model to the training set (EPOCHS, BATCHSIZE are hyperparametersdefined during the imports) -

In [28]:
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/10
50000/50000 [==============================] - 40s 809us/step - loss: 1.8551 - acc: 0.3213
Epoch 2/10
23744/50000 [=============>................] - ETA: 19s - loss: 1.4504 - acc: 0.4744

50000/50000 [==============================] - 37s 735us/step - loss: 1.3814 - acc: 0.5005
Epoch 3/10
40448/50000 [=======================>......] - ETA: 6s - loss: 1.1608 - acc: 0.5877

50000/50000 [==============================] - 37s 733us/step - loss: 1.1457 - acc: 0.5929
Epoch 4/10
48448/50000 [============================>.] - ETA: 1s - loss: 0.9986 - acc: 0.6455

50000/50000 [==============================] - 37s 743us/step - loss: 0.9949 - acc: 0.6470
Epoch 5/10
49728/50000 [============================>.] - ETA: 0s - loss: 0.8824 - acc: 0.6895

50000/50000 [==============================] - 38s 750us/step - loss: 0.8819 - acc: 0.6897
Epoch 6/10
50000/50000 [==============================] - 37s 737us/step - loss: 0.7960 - acc: 0.7186
Epoch 7/10
  640/50000 [..............................] - ETA: 35s - loss: 0.6401 - acc: 0.7734

50000/50000 [==============================] - 36s 726us/step - loss: 0.7261 - acc: 0.7435
Epoch 8/10
26560/50000 [==============>...............] - ETA: 18s - loss: 0.6588 - acc: 0.7681

50000/50000 [==============================] - 38s 760us/step - loss: 0.6627 - acc: 0.7668
Epoch 9/10
41472/50000 [=======================>......] - ETA: 6s - loss: 0.6060 - acc: 0.7843

50000/50000 [==============================] - 37s 735us/step - loss: 0.6052 - acc: 0.7853
Epoch 10/10
48192/50000 [===========================>..] - ETA: 1s - loss: 0.5525 - acc: 0.8034

50000/50000 [==============================] - 37s 731us/step - loss: 0.5535 - acc: 0.8031


Evaluating the model-

In [29]:
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 2s 176us/step

Test result: 77.070 loss: 0.661


Saving the model as *model.h5* to directory-

In [0]:
model_json = model.to_json()

with open('model.json', 'w') as json_file:
    json_file.write(model_json)
    
model.save_weights('model.h5')    

You can download the *model.h5* file-

In [0]:
files.download('model.h5')

#Conclusion

There you go! You have succefully compiled a keras CNN model using the free K80 gpu!

This model was not a complex one ofcourse you have not utilised the full powers of the K80 gpu, 

feel free to check out my repository on making a more complex model to get a better test results on the CIFAR-10 dataset using and more regularization techniques and some data augmentation to increase the training set size!

Here's the link:

Thank you for checking out this tutorial! : )